# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [3]:
#Importando as bibliotecas que serao utilizadas
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import math

In [4]:
#Lendo os dados
dados = pd.read_excel('tweets_starbucks_201809061627_ok (1).xlsx')
dados = dados.drop(columns=['Legenda'])

dados2 = pd.ExcelFile('tweets_starbucks_201809061627_ok (1).xlsx')

teste = pd.read_excel(dados2, 'Teste')
lista= teste["Teste"].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'tweets_starbucks_201809061627_ok (1).xlsx'

In [ ]:
#Criando as listas em função da classificação
mi = dados[dados.Classificação == 0]
i = dados[dados.Classificação == 1]
r = dados[dados.Classificação == 2]
mr= dados[dados.Classificação == 3]

In [ ]:
#Calculando a probilidade de um tweet estar dentro de uma determinada classificação
pb_mi=len(dados[dados.Classificação == 0])/len(dados["Treinamento"].tolist())
pb_i=len(dados[dados.Classificação == 1])/len(dados["Treinamento"].tolist())
pb_r=len(dados[dados.Classificação == 2])/len(dados["Treinamento"].tolist())
pb_mr=len(dados[dados.Classificação == 3])/len(dados["Treinamento"].tolist())
print('Probabilidade de ser muito irrelevante {0:.2f}%'.format(pb_mi*100))
print('Probabilidade de ser irrelevante {0:.2f}%'.format(pb_i*100))
print('Probabilidade de ser relevante {0:.2f}%'.format(pb_r*100))
print('Probabilidade de ser muito relevante {0:.2f}%'.format(pb_mr*100))

In [ ]:
#Criando os dicionarios de cada classificação (muito irrelevante, irrelevante, relevante e muito irrelevante), dentro de cada um desses dicionarios é adicionado cada plavra do grupo e seu respectivo numero de aparições
d_mi={}
d_i={}
d_r={}
d_mr={}
d_total = {}

for tweet in mi['Treinamento']:
    palavras=tweet.split()
    for p in palavras:
        if not p in d_mi:
            d_mi[p]=0
        d_mi[p]+=1

for tweet in i['Treinamento']:
    palavras=tweet.split()
    for p in palavras:
        if not p in d_i:
            d_i[p]=0
        d_i[p]+=1
        
for tweet in r['Treinamento']:
    palavras=tweet.split()
    for p in palavras:
        if not p in d_r:
            d_r[p]=0
        d_r[p]+=1

for tweet in mr['Treinamento']:
    palavras=tweet.split()
    for p in palavras:
        if not p in d_mr:
            d_mr[p]=0
        d_mr[p]+=1
        
for tweet in dados['Treinamento']:
    palavras=tweet.split()
    for p in palavras:
        if not p in d_total:
            d_total[p]=0
        d_total[p]+=1

In [ ]:
#Função que calcula a quantidade de repetições de cada grupo classificado
def soma_valores(d):
    soma = 0
    for e in d:
        soma += d[e]
    return soma

soma_mi = soma_valores(d_mi)
soma_i = soma_valores(d_i)
soma_r = soma_valores(d_r)
soma_mr = soma_valores(d_mr)

num_palavras_diferentes = len(d_total)

In [ ]:
#Calculando a probabilidade de cada tweet aplicando o Laplace smoothing
d_prob_mi={}
d_prob_i={}
d_prob_r={}
d_prob_mr={}

for e in d_mi:
    d_prob_mi[e]=(d_mi[e]+1)/(soma_mi+num_palavras_diferentes)

for e in d_i:
    d_prob_i[e]=(d_i[e]+1)/(soma_i+num_palavras_diferentes)

for e in d_r:   
    d_prob_r[e]=(d_r[e]+1)/(soma_r+num_palavras_diferentes)

for e in d_mr:
    d_prob_mr[e]=(d_mr[e]+1)/(soma_mr+num_palavras_diferentes)

In [ ]:
#Nessa etapa foi feita a classificação de cada tweet
import numpy as np
import operator

lista = teste["Teste"].tolist()

p_mi=[]
p_i=[]
p_r=[]
p_mr=[]
teste_classificacao=[]
for jj in lista:
    a=jj.split()
    for s in a:
        if s not in d_prob_mi:
            prob = 1/(soma_mi + num_palavras_diferentes)
        else:
            prob = (d_prob_mi[s]+1)/(soma_mi+num_palavras_diferentes)
        p_mi.append(prob)

        if s not in d_prob_i:
            prob = 1/(soma_i + num_palavras_diferentes)
        else:
            prob = (d_prob_i[s]+1)/(soma_i+num_palavras_diferentes)
        p_i.append(prob)

        if s not in d_prob_r:
            prob = 1/(soma_r + num_palavras_diferentes)
        else:
            prob = (d_prob_r[s]+1)/(soma_r+num_palavras_diferentes)
        p_r.append(prob)

        if s not in d_prob_mr:
            prob = 1/(soma_mr + num_palavras_diferentes)
        else:
            prob = (d_prob_mr[e]+1)/(soma_mr+num_palavras_diferentes)
        p_mr.append(prob)

        pf_mi=np.prod(p_mi)*pb_mi
        pf_i=np.prod(p_i)*pb_i
        pf_r=np.prod(p_r)*pb_r
        pf_mr=np.prod(p_mr)*pb_mr
        
        lista_probs=[pf_mi,pf_i,pf_r,pf_mr]
        max_valor=lista_probs[0]
        indice=0
        for i in range(len(lista_probs)):
            if lista_probs[i] > max_valor:
                max_valor=lista_probs[i]
                indice = i 
        maior_prob=lista_probs[i]
    teste_classificacao.append(maior_prob) 

In [ ]:
#Calculando a acurácia do programa
coluna1 = teste["Classificação"].tolist()
tabela = {"Classificação": coluna1,'Teste de Classificação':teste_classificacao}
tabela = pd.DataFrame(tabela)
acertos=0
i=0
while i < len(coluna1):
    if coluna1[i]==teste_classificacao[i]:
        acertos+=1
    i+=1
print("Porcentagem de acerto = {0} %".format(acertos/len(coluna1)*100))

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [ ]:
#Verificando a performance
s=0
mi_v=0
while s < len(coluna1):
    if coluna1[s]==0 and teste_classificacao[s]==0:
        mi_v+=1
    s+=1
porcentagem_mi_v=mi_v*100/len(coluna1)

s=0
mi_f=0
while s < len(coluna1):
    if coluna1[s]!=0 and teste_classificacao[s]==0:
        mi_f+=1
    s+=1
porcentagem_mi_f=mi_f*100/len(coluna1)

s=0
i_v=0
while s < len(coluna1):
    if coluna1[s]==1 and teste_classificacao[s]==1:
        i_v+=1
    s+=1
porcentagem_i_v=i_v*100/len(coluna1)

s=0
i_f=0
while s < len(coluna1):
    if coluna1[s]!=1 and teste_classificacao[s]==1:
        i_f+=1
    s+=1
porcentagem_i_f=i_f*100/len(coluna1)

s=0
r_v=0
while s < len(coluna1):
    if coluna1[s]==2 and teste_classificacao[s]==2:
        r_v+=1
    s+=1
porcentagem_r_v=r_v*100/len(coluna1)

s=0
r_f=0
while s < len(coluna1):
    if coluna1[s]!=2 and teste_classificacao[s]==2:
        r_f+=1
    s+=1
porcentagem_r_f=r_f*100/len(coluna1)

s=0
mr_v=0
while s < len(coluna1):
    if coluna1[s]==3 and teste_classificacao[s]==3:
        mr_v+=1
    s+=1
porcentagem_mr_v=mr_v*100/len(coluna1)

s=0
mr_f=0
while s < len(coluna1):
    if coluna1[s]!=3 and teste_classificacao[s]==3:
        mr_f+=1
    s+=1
porcentagem_mr_f=mr_f*100/len(coluna1)
print("Porcentagem de muito irrelevantes verdadeiros = {0} %".format(porcentagem_mi_v))
print("Porcentagem de muito irrelevantes falsos = {0} %".format(porcentagem_mi_f))
print("Porcentagem de irrelevantes verdadeiros = {0} %".format(porcentagem_i_v))
print("Porcentagem de irrelevantes falsos = {0} %".format(porcentagem_i_f))
print("Porcentagem de relevantes verdadeiros = {0} %".format(porcentagem_r_v))
print("Porcentagem de relevantes falsos = {0} %".format(porcentagem_r_f))
print("Porcentagem de muito relevantes verdadeiros = {0} %".format(porcentagem_mr_v))
print("Porcentagem de muito relevantes falsos = {0} %".format(porcentagem_mr_f))

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
